# Create 3d clone of the scene

In [74]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [75]:
def convert_to_homogeneous(points):
    n = points.shape[0]
    return np.concatenate((points, np.ones((n,1))), axis=1)

In [76]:
def convert_to_cartesian(points):
    return points[:,:-1]

In [77]:
def normalize_and_homogenize_2d(points):
    centroid = np.mean(points, axis=0)
    centred_points = points - centroid
    scale = np.sqrt(2) / np.mean(np.linalg.norm(centred_points, axis=1))
    T = np.array([[scale, 0, -centroid[0] * scale],
                  [0, scale, -centroid[1] * scale],
                  [0, 0, 1]])
    points = np.dot(T, convert_to_homogeneous(points).T).T
    return points, T

In [78]:
def normalize_and_homogenize_3d(points):
    centroid = np.mean(points, axis=0)
    centred_points = points - centroid
    scale = np.sqrt(3) / np.mean(np.linalg.norm(centred_points, axis=1))
    T = np.array([[scale, 0, 0, -centroid[0] * scale],
                  [0, scale, 0, -centroid[1] * scale],
                  [0, 0, scale, -centroid[2] * scale],
                  [0, 0, 0, 1]])
    points = np.dot(T, convert_to_homogeneous(points).T).T
    return points, T

In [103]:
points_3d = np.array([[463, 925,   0],
                      [412, 925,   0],
                      [412, 599,   0],
                      [463, 925, 120],
                      [412, 925, 120],
                      [412, 599, 120],
                      [338, 919,   0],
                      [291, 923,   0],
                      [338, 919, 120],
                      [291, 923, 120]])
points_3d, T = normalize_and_homogenize_3d(points_3d)
points_3d = convert_to_cartesian(points_3d)
print(points_3d)
print(np.mean(np.linalg.norm(points_3d, axis=1)))
print(np.mean(points_3d, axis=0))

[[ 0.9784034   0.81901437 -0.73564165]
 [ 0.35310799  0.81901437 -0.73564165]
 [ 0.35310799 -3.17797193 -0.73564165]
 [ 0.9784034   0.81901437  0.73564165]
 [ 0.35310799  0.81901437  0.73564165]
 [ 0.35310799 -3.17797193  0.73564165]
 [-0.55418338  0.74545021 -0.73564165]
 [-1.130436    0.79449298 -0.73564165]
 [-0.55418338  0.74545021  0.73564165]
 [-1.130436    0.79449298  0.73564165]]
1.7320508075688763
[ 5.32907052e-16 -1.24344979e-15  0.00000000e+00]


In [104]:
points_2d = np.array([[1932, 1314],
                      [1556, 1332],
                      [1412,  524],
                      [2071,  772],
                      [1619,  764],
                      [1432,  257],
                      [ 942, 1355],
                      [ 546, 1336],
                      [ 760,  777],
                      [ 407,  803]])
points_2d, T = normalize_and_homogenize_2d(points_2d)
print(convert_to_cartesian(points_2d))
print(np.mean(np.linalg.norm(points_2d[:,:2], axis=1)))
print(np.mean(points_2d[:,:2], axis=0))

[[ 1.4784385   0.86930314]
 [ 0.64162851  0.9093632 ]
 [ 0.32114809 -0.88888806]
 [ 1.78779113 -0.33694955]
 [ 0.7818387  -0.35475402]
 [ 0.36565926 -1.48311217]
 [-0.7248644   0.96055104]
 [-1.60618556  0.91826543]
 [-1.12991604 -0.32582176]
 [-1.91553819 -0.26795724]]
1.4142135623730951
[-4.4408921e-17  4.4408921e-17]


In [105]:
n = points_2d.shape[0]
u = points_2d[:,0]
v = points_2d[:,1]
X = points_3d[:,0]
Y = points_3d[:,1]
Z = points_3d[:,2]
A = np.zeros((2*n, 12))

In [121]:
for i in range(n):
    A[2*i,:] = [X[i], Y[i], Z[i], 1, 0, 0, 0, 0, -u[i]*X[i], -u[i]*Y[i], -u[i]*Z[i], -u[i]]
    A[2*i+1,:] = [0, 0, 0, 0, X[i], Y[i], Z[i], 1, -v[i]*X[i], -v[i]*Y[i], -v[i]*Z[i], -v[i]]

## Calculation 1

In [129]:
U, S, V = np.linalg.svd(A)

In [130]:
P = V[:, -1].reshape((3, 4))

In [131]:
P

array([[-0.01497537, -0.66953079,  0.41951313, -0.18695756],
       [-0.17632161, -0.12999335,  0.00965445,  0.02126225],
       [-0.0015472 ,  0.01127641, -0.09949918, -0.53103423]])

In [143]:
from scipy.optimize import least_squares

def residuals(P, points_2d, points_3d):
    X = P.reshape(3, 4)
    projected = (X @ points_3d.T).T
    projected = projected / projected[:, -1:]
    return np.ravel(points_2d - projected)

In [144]:

res = least_squares(residuals, P.flatten(), args=(convert_to_homogeneous(points_2d), convert_to_homogeneous(points_3d)))
X_opt = res.x.reshape(-1, 3)

ValueError: operands could not be broadcast together with shapes (10,4) (10,3) 

In [137]:
p = optimize_projection_matrix(P, points_3d, points_2d)

ValueError: could not broadcast input array from shape (10,4) into shape (10,8)

In [125]:
points_3d_h = convert_to_homogeneous(points_3d)
points_2d_h = np.dot(P, points_3d_h.T)
points_2d_h = points_2d_h / points_2d_h[2,:]
print(points_2d_h.T)
print(points_2d)

[[ 2.35178146  0.58833636  1.        ]
 [ 2.33599879  0.3441327   1.        ]
 [-3.29183388 -0.73856579  1.        ]
 [ 0.73989512  0.42013773  1.        ]
 [ 0.72537351  0.23568919  1.        ]
 [-3.50307897 -0.59196341  1.        ]
 [ 2.19889998 -0.03337451  1.        ]
 [ 2.26014072 -0.24646393  1.        ]
 [ 0.62045338 -0.04903705  1.        ]
 [ 0.66274729 -0.20960634  1.        ]]
[[ 1.4784385   0.86930314  1.        ]
 [ 0.64162851  0.9093632   1.        ]
 [ 0.32114809 -0.88888806  1.        ]
 [ 1.78779113 -0.33694955  1.        ]
 [ 0.7818387  -0.35475402  1.        ]
 [ 0.36565926 -1.48311217  1.        ]
 [-0.7248644   0.96055104  1.        ]
 [-1.60618556  0.91826543  1.        ]
 [-1.12991604 -0.32582176  1.        ]
 [-1.91553819 -0.26795724  1.        ]]


## Calculation 2

In [51]:
#camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(points_3d, points_2d, [2592, 1944], None, None)

error: OpenCV(4.6.0) /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/calib3d/src/calibration.cpp:3350: error: (-210:Unsupported format or combination of formats) objectPoints should contain vector of vectors of points of type Point3f in function 'collectCalibrationData'


## Calculation 3

In [98]:
res = np.linalg.lstsq(A, b, rcond=None)

In [99]:
m = res[0]

In [100]:
M = m.reshape((3,4))

In [101]:
print(M)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [102]:
points_3d_h = np.concatenate((points_3d, np.ones((n,1))), axis=1)
points_2d_h = np.dot(M, points_3d_h.T)
points_2d_h = points_2d_h / points_2d_h[2,:]
print(points_2d_h.T)
print(points_2d)

[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[ 1.4784385   0.86930314  1.        ]
 [ 0.64162851  0.9093632   1.        ]
 [ 0.32114809 -0.88888806  1.        ]
 [ 1.78779113 -0.33694955  1.        ]
 [ 0.7818387  -0.35475402  1.        ]
 [ 0.36565926 -1.48311217  1.        ]
 [-0.7248644   0.96055104  1.        ]
 [-1.60618556  0.91826543  1.        ]
 [-1.12991604 -0.32582176  1.        ]
 [-1.91553819 -0.26795724  1.        ]]


/var/folders/8_/51vdws854m1218_v37ctyqj80000gn/T/ipykernel_28574/1025730394.py:3: RuntimeWarning: invalid value encountered in divide
  points_2d_h = points_2d_h / points_2d_h[2,:]


In [1]:
#normalisierung

In [ ]:
#pseudo inverse